## **Setting up the Ray cluster**

**Let's first login to the OpenShift cluster and navigate to the project**

In [1]:
! oc login --token=your-token --server=https://api.cluster-c4lhj.c4lhj.sandbox1290.opentlc.com:6443 --insecure-skip-tls-verify

Logged into "https://api.cluster-c4lhj.c4lhj.sandbox1290.opentlc.com:6443" as "opentlc-mgr" using the token provided.

You have access to 101 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "default".


In [2]:
! oc project default

Already on project "default" on server "https://api.cluster-c4lhj.c4lhj.sandbox1290.opentlc.com:6443".


**We will import the CodeFlare pieces from codflare-sdk**

In [3]:
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration

### **Request aggregated resources using CodeFlare**

**cluster-up() will create an AppWrapper CRD that will request aggregated resources and create
a Ray cluster with Ray head and two Ray worker nodes (each represented by a pod) when resources are available. If resources are not available,
it will wait in a queue and instantly deploy the Ray cluster when resources free up.**

In [4]:
# Create our cluster and submit appwrapper
cluster = Cluster(ClusterConfiguration(name='road-ray', min_worker=1, max_worker=1, min_cpus=2, max_cpus=2, min_memory=8, max_memory=8, gpu=0))

Written to: road-ray.yaml


In [5]:
cluster.up()

In [6]:
cluster.is_ready()

╭─────────────────────────────────╮
│  🚀 List of CodeFlare clusters  │
│           in queue🚀            │
│ +----------+---------+--------+ │
│ |          | Name    | Status | │
│ +==========+=========+========+ │
│ |          | pending |        | │
│ | road-ray |         |        | │
│ +----------+---------+--------+ │
╰─────────────────────────────────╯

(False, <CodeFlareClusterStatus.QUEUED: 2>)

In [8]:
cluster.status()

                  🚀 List of CodeFlare clusters 🚀                 
                                                                   
 ╭───────────────────────────────────────────────────────────────╮ 
 │   Owner                                                       │ 
 │   road-ray                                        Active ✅   │ 
 │                                                               │ 
 │   URI: ray://road-ray-head-svc.default.svc:10001              │ 
 │                                                               │ 
 │   ]8;id=994463;https://codeflare-raydashboard.research.ibm.com?rayclustername=road-ray\Dashboard🔗]8;;\                                                 │ 
 │                                                               │ 
 │                      Cluster Resources                        │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮    │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │    │ 
 │   │            │  │                                      │    │ 
 │   │  1    1    │  │  8G~8G       2           0           │    │ 
 │   │            │  │                                      │    │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯    │ 
 ╰───────────────────────────────────────────────────────────────╯

<RayClusterStatus.READY: 'ready'>

In [9]:
ray_cluster_uri = cluster.cluster_uri()

**Below we will go ahead and connect to this cluster so that we can run our code on it.**

In [10]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for this training
runtime_env = {"pip": ["scikit-learn"]}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env)

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


## Load Data

In [11]:
#!pip install -r requirements.txt
#import joblib
import pandas as pd

df = pd.read_csv('road_roughness_data.csv')
print(df)

           acc_x     acc_y     acc_z     speed    gyro_x    gyro_y    gyro_z  \
0       0.365116  0.167893  9.793961  0.009128 -0.133896 -0.018883  0.138092   
1       0.392649  0.176273  9.771216  0.009128 -0.027084 -0.003624  0.000763   
2       0.409408  0.181062  9.732909  0.009128  0.125504 -0.186729 -0.090790   
3       0.371101  0.164302  9.749668  0.009128 -0.088120 -0.034142  0.046539   
4       0.390255  0.159514  9.869378  0.009128 -0.179672  0.118446 -0.182343   
...          ...       ...       ...       ...       ...       ...       ...   
144031 -0.527921 -0.322918  9.583271  0.005715 -0.332260 -0.095177 -0.060272   
144032 -0.663194 -0.575506  9.433633  0.005715 -0.240707 -0.308800 -0.182343   
144033 -0.375890 -0.245106  9.957964  0.005715  0.064468 -0.156212  0.000763   
144034 -0.385466 -0.091877  9.840648  0.005715 -0.423813  0.820351 -0.243378   
144035 -0.575805 -0.502483  9.457575  0.005715 -0.500107  0.057411 -0.060272   

        road_condition  
0             

## Features

In [12]:
df.iloc[:,:-1]

,acc_x,acc_y,acc_z,speed,gyro_x,gyro_y,gyro_z
0,0.365116,0.167893,9.793961,0.009128,-0.133896,-0.018883,0.138092
1,0.392649,0.176273,9.771216,0.009128,-0.027084,-0.003624,0.000763
2,0.409408,0.181062,9.732909,0.009128,0.125504,-0.186729,-0.090790
3,0.371101,0.164302,9.749668,0.009128,-0.088120,-0.034142,0.046539
4,0.390255,0.159514,9.869378,0.009128,-0.179672,0.118446,-0.182343
...,...,...,...,...,...,...,...
144031,-0.527921,-0.322918,9.583271,0.005715,-0.332260,-0.095177,-0.060272
144032,-0.663194,-0.575506,9.433633,0.005715,-0.240707,-0.308800,-0.182343
144033,-0.375890,-0.245106,9.957964,0.005715,0.064468,-0.156212,0.000763
144034,-0.385466,-0.091877,9.840648,0.005715,-0.423813,0.820351,-0.243378


## Target variable

In [13]:
df.iloc[:,-1:]

,road_condition
0,2
1,2
2,2
3,2
4,2
...,...
144031,2
144032,2
144033,2
144034,2


## Split data into Train and Test sets

In [14]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1]
#y = df.iloc[:,-1:]
y = df['road_condition']

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80) # 70% training and 30% test

In [15]:
# Create Ray object references
X_train_remote, X_test_remote, y_train_remote, y_test_remote = ray.put(X_train), ray.put(X_test), ray.put(y_train), ray.put(y_test)

## Fit Random Forest Classifier to Train set and Run prediction on test data

In [16]:
@ray.remote
def train_fn(X_train, y_train, X_test):
    #Import Random Forest Model
    from sklearn.ensemble import RandomForestClassifier

    #Create a Gaussian Classifier
    clf = RandomForestClassifier(n_estimators=100)

    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train,y_train)
    
    #Run prediction on test data and return the results
    y_pred = clf.predict(X_test)
    return y_pred

In [17]:
y_pred = ray.get(train_fn.remote(X_train_remote, y_train_remote, X_test_remote))

**Let's clean up. cluster.down() will delete the Ray cluster, free up resources and delete the AppWrapper CRD.**

In [ ]:
cluster.down()

## Test model accuracy

In [18]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8903748188390075


## Save model

In [ ]:
# save the model to disk
filename = 'road-model.joblib'
joblib.dump(clf, filename)

## Load and Test prediction from saved model

In [ ]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

In [ ]:
!nvidia-smi